In [1]:
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/research'

In [2]:
import os

os.chdir('../')
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf'

In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

#### Training Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class CallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    model_checkpoint_filepath: Path

@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

#### Configuration Manager - Training

In [6]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utilities import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_callbacks_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.model_checkpoint_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_log_dir)
        ])

        callbacks_config = CallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_log_dir=Path(config.tensorboard_log_dir),
            model_checkpoint_filepath=Path(config.model_checkpoint_filepath)
        )

        return callbacks_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        base_model = self.config.base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "images")

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(base_model.updated_model_path),
            training_data=Path(training_data),
            params_batch_size=params.BATCH_SIZE,
            params_epochs=params.EPOCHS,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config

In [ ]:
print(tf.__version__)
print("GPUs Available:", tf.config.list_physical_devices('GPU'))

2.18.0
GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/cnn_venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [7]:
%pip show tensorflow tensorflow-metal | grep -E "^(Name|Version):"

Name: tensorflow
Version: 2.18.0
Name: tensorflow-metal
Version: 1.2.0
Note: you may need to restart the kernel to use updated packages.


#### Training Component

In [8]:
import tensorflow as tf
import time
import math

class Callbacks:
    def __init__(self, config=CallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.model_checkpoint_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

class Training:
    def __init__(self, config=TrainingConfig):
        self.config = config
        self.train_generator = None
        self.valid_generator = None
    
    def get_base_model(self):
        # Load WITHOUT optimizer/compile state
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path, compile=False)

        # Compile fresh optimizer
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"],
            steps_per_execution=10,
            jit_compile="auto"
        )
    
    def train_validation_generator(self):
        ds_kwargs = dict(
            directory=self.config.training_data,
            validation_split=0.20,
            seed=42,
            image_size=self.config.params_image_size[:-1],  # e.g., [224, 224]
            batch_size=self.config.params_batch_size,
            label_mode="categorical",
            interpolation="bilinear"
        )

        train_ds = tf.keras.utils.image_dataset_from_directory(
            subset="training",
            shuffle=True,
            **ds_kwargs
        )

        validation_ds = tf.keras.utils.image_dataset_from_directory(
            subset="validation",
            shuffle=False,
            **ds_kwargs
        )

        # Rescale
        layers = tf.keras.layers
        AUTOTUNE = tf.data.AUTOTUNE
        rescale = layers.Rescaling(1.0 / 255)
        
        # # Cache raw decoded images
        # train_ds = train_ds.cache()
        # validation_ds = validation_ds.cache()

        # Augmentation
        if self.config.params_is_augmentation:
            aug = tf.keras.Sequential([
                layers.RandomRotation(40 / 360),          # ~40 degrees
                layers.RandomFlip("horizontal"),
                layers.RandomTranslation(0.2, 0.2),       # height, width shift
                layers.RandomZoom(0.2, 0.2),
                layers.RandomShear(0.2, 0.2)              # shear transformations
            ])

            train_ds = train_ds.map(
                lambda x, y: (aug(rescale(x), training=True), y),
                num_parallel_calls=AUTOTUNE
            )
        else:
            train_ds = train_ds.map(
                lambda x, y: (rescale(x), y),
                num_parallel_calls=AUTOTUNE
            )
        
        validation_ds = validation_ds.map(
            lambda x, y: (rescale(x), y),
            num_parallel_calls=AUTOTUNE
        )

        # Performance
        train_ds = train_ds.cache().prefetch(AUTOTUNE)
        validation_ds = validation_ds.cache().prefetch(AUTOTUNE)

        # Assign to class attributes
        self.train_generator = train_ds
        self.valid_generator = validation_ds

        # Save number of images for steps calculation
        self.num_train_images = int((1 - ds_kwargs['validation_split']) * 8067)  # 80% split
        self.num_val_images = int(ds_kwargs['validation_split'] * 8067)

    def train(self, callback_list: list):
        self.steps_per_epoch = math.ceil(self.num_train_images / self.config.params_batch_size)
        self.validation_steps = math.ceil(self.num_val_images / self.config.params_batch_size)
    
        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

#### Training Pipeline

In [9]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()
    prepare_callbacks = Callbacks(config=callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_validation_generator()
    training.train(callback_list=callback_list)
except Exception as e:
    raise e

[2025-12-22 19:01:33,814]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-22 19:01:33,818]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-22 19:01:33,820]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts
[2025-12-22 19:01:33,821]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/callbacks/checkpoint_dir
[2025-12-22 19:01:33,821]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/callbacks/tensorboard_log_dir
[2025-12-22 19:01:33,823]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/training


2025-12-22 19:01:33.842871: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-12-22 19:01:33.842906: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-12-22 19:01:33.842917: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.66 GB
I0000 00:00:1766451693.842954   94422 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1766451693.843017   94422 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 8067 files belonging to 4 classes.
Using 6454 files for training.
Found 8067 files belonging to 4 classes.
Using 1613 files for validation.
Epoch 1/10


2025-12-22 19:01:34.759968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


110/101 ━━━━━━━━━━━━━━━━━━━━━ -3s 436ms/step - accuracy: 0.5949 - loss: 8.3089[2025-12-22 19:02:34,372]: absl - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
101/101 ━━━━━━━━━━━━━━━━━━━━ 60s 592ms/step - accuracy: 0.6842 - loss: 4.6541 - val_accuracy: 0.6652 - val_loss: 2.2184
Epoch 2/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 57s 560ms/step - accuracy: 0.8210 - loss: 0.8284 - val_accuracy: 0.6045 - val_loss: 2.8660
Epoch 3/10
110/101 ━━━━━━━━━━━━━━━━━━━━━ -3s 405ms/step - accuracy: 0.8229 - loss: 0.8600[2025-12-22 19:04:26,881]: absl - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format